<a href="https://colab.research.google.com/github/AbhinavKaintura/PRODIGY_GA_02/blob/main/Task2Completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from diffusers import StableDiffusionPipeline
import torch
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader, Dataset
import numpy as np
import cv2  # OpenCV for efficient image saving
import time

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
# Load Stable Diffusion with mixed precision (FP16) for efficient GPU usage
pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

...

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
# Function to prune the unet model in Stable Diffusion pipeline
def prune_model(unet, amount=0.2):
    for module in unet.modules():
        if isinstance(module, torch.nn.Conv2d) or isinstance(module, torch.nn.Linear):
            prune.l1_unstructured(module, name="weight", amount=amount)
    return unet

# Prune the UNet model in Stable Diffusion pipeline
pipe.unet = prune_model(pipe.unet, amount=0.2)

In [5]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use the GPU
    print("CUDA is available, using GPU")
else:
    device = torch.device("cpu")  # Fall back to the CPU
    print("CUDA is not available, using CPU")
print(torch.cuda.get_device_name(0))  # Displays the name of the GPU

CUDA is available, using GPU
Tesla T4


In [6]:

# Custom Dataset class to handle prompt inputs
class PromptDataset(Dataset):
    def __init__(self, prompts):
        self.prompts = prompts

    def __len__(self):
        return len(self.prompts)

    def __getitem__(self, idx):
        return self.prompts[idx]

In [7]:

# Function to generate and save images in batch
def generate_images(prompts, batch_size=2):
    prompt_dataset = PromptDataset(prompts)

    # Set num_workers=0 to avoid multiprocessing issues
    prompt_loader = DataLoader(prompt_dataset, batch_size=batch_size, num_workers=0, pin_memory=True)

    for batch_num, batch in enumerate(prompt_loader):
        start_time = time.time()  # Track batch generation time
        images = pipe(batch).images  # Generate a batch of images

        # Save images using OpenCV for efficient processing
        for i, image in enumerate(images):
            img_num = batch_num * batch_size + i + 1
            cv_image = np.array(image)[:, :, ::-1]  # Convert RGB to BGR for OpenCV
            cv2.imwrite(f"optimized_image_{img_num}.png", cv_image)

        print(f"Batch {batch_num+1} generated in {time.time() - start_time:.2f} seconds.")


In [9]:

# User input for prompts and batch size
prompts = [
    "a futuristic cityscape at sunset",
    "a mountain landscape with clouds",
    "a serene lake at dawn with reflections",
    "a robot walking through a neon-lit street"
]
batch_size = 2

In [10]:
# Generate and save images
generate_images(prompts, batch_size)

  0%|          | 0/50 [00:00<?, ?it/s]

Batch 1 generated in 16.43 seconds.


  0%|          | 0/50 [00:00<?, ?it/s]

Batch 2 generated in 14.11 seconds.
